# Rerun the models, but with AQI

In [ ]:
import pandas
from sklearn.ensemble import RandomForestRegressor
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import numpy as np
from xgboost import XGBRegressor
import tensorflow as tf
%run -i ./Model_Eval.ipynb
from keras.callbacks import EarlyStopping
tf.get_logger().setLevel('INFO')
tf.autograph.set_verbosity(0)
import warnings
warnings.filterwarnings('ignore')

In [2]:
training_data = pd.read_csv('../Datasets/training_data_with_aqi.csv').sample(frac=1)

In [3]:
# import from Model_Eval.ipynb
numerical_features = get_numerical_features(training_data)

X = training_data[numerical_features]
Y = training_data['DIABETES_3Y_Change_Percentage']

In [4]:
# try random forest regressor
rf = RandomForestRegressor(
    # hyper parameters
    max_depth=100,
    max_features='sqrt',
    n_estimators=1000,
    # train params
    n_jobs=16,
    verbose=0
)

rf_r2_scores = cross_val_score(rf, X, Y, cv=5, scoring='r2')
print(rf_r2_scores)

[0.21184167 0.2690439  0.24393383 0.28159868 0.23095976]


In [5]:
np.mean(rf_r2_scores)

0.24747556456701303

In [14]:
xgb_model = XGBRegressor(
    max_depth=3,
    eta=0.1,
    n_estimators=1500
)

xgb_r2_scores = cross_val_score(xgb_model, X, Y, cv=5, scoring='r2')

In [15]:
np.mean(xgb_r2_scores)

0.22109801800276568

In [ ]:
%run -i Cross_Val_Util.ipynb
def build_model(normalizer):
    model = tf.keras.Sequential([
        normalizer,
        tf.keras.layers.Dense(50, activation='relu'),
        tf.keras.layers.Dense(45, activation='relu'),
        tf.keras.layers.Dense(5, activation='relu'),
        tf.keras.layers.Dense(1, activation='linear')
    ])

    model.compile(
        optimizer="adam",
        loss="mean_squared_error"
    )
    return model

fit_params = {
    'batch_size': 16,
    'epochs': 50,
    'callbacks': [EarlyStopping(monitor='val_loss', patience=3)],
    'verbose': 0
}

nn_r2_scores = cv_tf_nn(build_model, X, Y, fit_params=fit_params, cv=5)

 60%|████████████████████████████████████████████████████████████████████▍                                             | 3/5 [08:57<05:52, 176.05s/it]

In [17]:
np.mean(nn_r2_scores)

0.23324313703180244